<a href="https://colab.research.google.com/github/ji-hun-choi/SOTA-Study/blob/main/Transferlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time
import copy
import numpy as np
import matplotlib.pyplot as plt
import torch

import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, models, transforms


## Data set

In [ ]:
import gdown
## google drive에서 압축된 dataset download
url = 'https://drive.google.com/uc?id=1DtxpOUpARxcnU4qHINz9OgiOdXiji1RZ'
fname = 'hym_data.zip'
gdown.download(url, fname, quiet=False)
!unzip hym_data.zip # test를 위한 data.zip 파일 가져오기.

Downloading...
From: https://drive.google.com/uc?id=1DtxpOUpARxcnU4qHINz9OgiOdXiji1RZ
To: /content/hym_data.zip
100%|██████████| 95.0M/95.0M [00:01<00:00, 68.0MB/s]


Archive:  hym_data.zip
  inflating: hym_data/val/ants/183260961_64ab754c97.jpg  
  inflating: hym_data/val/ants/desert_ant.jpg  
  inflating: hym_data/val/ants/Ant-1818.jpg  
  inflating: hym_data/val/ants/436944325_d4925a38c7.jpg  
  inflating: hym_data/val/ants/768870506_8f115d3d37.jpg  
  inflating: hym_data/val/ants/412436937_4c2378efc2.jpg  
  inflating: hym_data/val/ants/11381045_b352a47d8c.jpg  
  inflating: hym_data/val/ants/649407494_9b6bc4949f.jpg  
  inflating: hym_data/val/ants/119785936_dd428e40c3.jpg  
  inflating: hym_data/val/ants/159515240_d5981e20d1.jpg  
  inflating: hym_data/val/ants/1119630822_cd325ea21a.jpg  
  inflating: hym_data/val/ants/172772109_d0a8e15fb0.jpg  
  inflating: hym_data/val/ants/540543309_ddbb193ee5.jpg  
  inflating: hym_data/val/ants/F.pergan.28(f).jpg  
  inflating: hym_data/val/ants/2255445811_dabcdf7258.jpg  
  inflating: hym_data/val/ants/459442412_412fecf3fe.jpg  
  inflating: hym_data/val/ants/263615709_cfb28f6b8e.jpg  
  inflating: hym_d

In [ ]:
ddir = '/content/hym_data'
batch_size = 4
num_workers = 2
data_transformers = {
    'train': transforms.Compose(
        [
         transforms.RandomResizedCrop(224), 
         transforms.RandomHorizontalFlip(),
         transforms.ToTensor(),
         transforms.Normalize([0.490, 0.449, 0.411], [0.231, 0.221, 0.230])
        ]
    ),
    'val': transforms.Compose(
        [
         transforms.Resize(256),
         transforms.CenterCrop(224),
         transforms.ToTensor(),
         transforms.Normalize([0.490, 0.449, 0.411],[0.231, 0.221, 0.230])
        ]
    )
}

img_data = {
    k:datasets.ImageFolder(os.path.join(ddir, k), data_transformers[k])
    for k in ['train', 'val']
}
dloaders = {
    k: torch.utils.data.DataLoader(
        img_data[k], batch_size=batch_size, shuffle=True, num_workers=num_workers
    )
    for k in ['train', 'val']
}
dset_sizes = {x: len(img_data[x]) for x in ['train', 'val']}
classes = img_data['train'].classes
dvc = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 학습

In [ ]:
def train(model, loss_func, optimizer, epochs=10):
    start = time.time()

    accuracy = 0.0

    for e in range(epochs):
        print(f'Epoch number {e}/{epochs - 1}')
        print("=" * 20)

        for dset in ['train', 'val']:
            if dset == 'train':
                model.train()
            else:
                model.eval()
            loss = 0.0
            successes = 0

            for imgs, tgts in dloaders[dset]:
                imgs = imgs.to(dvc)
                tgts = tgts.to(dvc)
                optimizer.zero_grad()

                with torch.set_grad_enabled(dset == 'train'):
                    ops = model(imgs)
                    _, preds = torch.max(ops, 1)
                    loss_curr = loss_func(ops, tgts)
                    
                    if dset == 'train':
                        loss_curr.backward()
                        optimizer.step()
                
                loss += loss_curr.item() * imgs.size(0)
                successes += torch.sum(preds == tgts.data)

            loss_epoch = loss / dset_sizes[dset]
            accuracy_epoch = successes.double() / dset_sizes[dset]

            print(f'{dset} loss in this epoch: {loss_epoch}, accuracy in this epoch: {accuracy_epoch}')
            if dset == 'val' and accuracy_epoch > accuracy:
                accuracy = accuracy_epoch
    time_delta = time.time() - start
    print(f'Training finished in {time_delta // 60}mins {time_delta % 60} secs')
    print(f'Best validation set acuuracy: {accuracy}')
    return model

## classifier 수정 및 전체 훈련

In [ ]:

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs, 2)

if torch.cuda.is_available() :
    model = model.cuda()

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
train(model, loss_func, optimizer, epochs=5)

Epoch number 0/4
train loss in this epoch: 0.4990538439911897, accuracy in this epoch: 0.7459016393442623
val loss in this epoch: 0.35399045407360424, accuracy in this epoch: 0.823529411764706
Epoch number 1/4
train loss in this epoch: 0.43541875695351695, accuracy in this epoch: 0.819672131147541
val loss in this epoch: 0.24825544258445695, accuracy in this epoch: 0.869281045751634
Epoch number 2/4
train loss in this epoch: 0.423230471211623, accuracy in this epoch: 0.8155737704918034
val loss in this epoch: 0.39410589819810554, accuracy in this epoch: 0.8562091503267975
Epoch number 3/4
train loss in this epoch: 0.3140608478551272, accuracy in this epoch: 0.8770491803278689
val loss in this epoch: 0.20423847676431334, accuracy in this epoch: 0.9215686274509804
Epoch number 4/4
train loss in this epoch: 0.34841608933982304, accuracy in this epoch: 0.8319672131147542
val loss in this epoch: 0.2788605844614163, accuracy in this epoch: 0.8954248366013072
Training finished in 0.0mins 25.5

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## classifier 수정 및 fc만 훈련

In [ ]:
model_conv = models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

if torch.cuda.is_available() :
    model_conv = model_conv.cuda()

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_conv.parameters(), lr=0.0001)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
train(model_conv, loss_func, optimizer, epochs=5)

Epoch number 0/4
train loss in this epoch: 0.6900790836967405, accuracy in this epoch: 0.5573770491803279
val loss in this epoch: 0.6728399423212786, accuracy in this epoch: 0.5751633986928105
Epoch number 1/4
train loss in this epoch: 0.6301410046757244, accuracy in this epoch: 0.6557377049180328
val loss in this epoch: 0.593947327214908, accuracy in this epoch: 0.6993464052287582
Epoch number 2/4
train loss in this epoch: 0.6071628122056116, accuracy in this epoch: 0.7049180327868853
val loss in this epoch: 0.5327155408906001, accuracy in this epoch: 0.7516339869281046
Epoch number 3/4
train loss in this epoch: 0.5803893345301269, accuracy in this epoch: 0.7049180327868853
val loss in this epoch: 0.49601294246374394, accuracy in this epoch: 0.7843137254901961
Epoch number 4/4
train loss in this epoch: 0.5507350627516137, accuracy in this epoch: 0.7254098360655739
val loss in this epoch: 0.4788840218307146, accuracy in this epoch: 0.7908496732026145
Training finished in 0.0mins 12.640

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Conv 일부분, classifier 수정

In [ ]:
model_conv = models.resnet18(pretrained=True)
for name, param in model_conv.named_parameters():
    param.requires_grad = False
    if 'layer4.1' in name:
        param.requires_grad = True

num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

if torch.cuda.is_available() :
    model_conv = model_conv.cuda()

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_conv.parameters(), lr=0.0001)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
train(model_conv, loss_func, optimizer, epochs=5)

Epoch number 0/4
train loss in this epoch: 0.4944095739819964, accuracy in this epoch: 0.7540983606557378
val loss in this epoch: 0.2235846444964409, accuracy in this epoch: 0.934640522875817
Epoch number 1/4
train loss in this epoch: 0.34444403975102744, accuracy in this epoch: 0.8606557377049181
val loss in this epoch: 0.21803184906290873, accuracy in this epoch: 0.9150326797385622
Epoch number 2/4
train loss in this epoch: 0.4231836354879082, accuracy in this epoch: 0.8073770491803279
val loss in this epoch: 0.21834728937425646, accuracy in this epoch: 0.9281045751633987
Epoch number 3/4
train loss in this epoch: 0.346681987805689, accuracy in this epoch: 0.8647540983606558
val loss in this epoch: 0.18944422698585817, accuracy in this epoch: 0.9150326797385622
Epoch number 4/4
train loss in this epoch: 0.28751902636445936, accuracy in this epoch: 0.8729508196721312
val loss in this epoch: 0.20771154257913235, accuracy in this epoch: 0.9215686274509804
Training finished in 0.0mins 12

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  